In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

df = pd.read_csv("./preprocessing/reduced_features.csv")

X = df.drop("Label", axis=1).values
y = df["Label"].values

X = X.reshape(X.shape[0], X.shape[1], 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# CNN-LSTM model
class CNNLSTM(nn.Module):
    def __init__(self, input_size, cnn_out_channels, lstm_hidden_size, lstm_layers, output_size):
        super(CNNLSTM, self).__init__()
        
        # CNN layer
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=cnn_out_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=cnn_out_channels, hidden_size=lstm_hidden_size, num_layers=lstm_layers, batch_first=True)
        
        # Fully connected output layer
        self.fc = nn.Linear(lstm_hidden_size, output_size)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        # CNN part
        x = x.permute(0, 2, 1)  
        x = self.conv1d(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # LSTM part
        x = x.permute(0, 2, 1)
        lstm_out, _ = self.lstm(x)
        out = self.dropout(lstm_out[:, -1, :])
        
        out = self.fc(out)
        return out


input_size = X.shape[1]
cnn_out_channels = 32
lstm_hidden_size = 64
lstm_layers = 2
output_size = 1

model = CNNLSTM(input_size, cnn_out_channels, lstm_hidden_size, lstm_layers, output_size)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


model.eval()
with torch.no_grad():
    y_pred_logits = model(X_test_tensor).squeeze()
    y_pred = torch.sigmoid(y_pred_logits).round().numpy()

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("\nModel Evaluation Metrics:")
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, zero_division=0))


Epoch [10/50], Loss: 0.6885
Epoch [20/50], Loss: 0.6784
Epoch [30/50], Loss: 0.6358
Epoch [40/50], Loss: 0.5175
Epoch [50/50], Loss: 0.4211

Model Evaluation Metrics:
Accuracy: 78.95%
Precision: 0.86
Recall: 0.67
F1 Score: 0.75

Confusion Matrix:
[[9 1]
 [3 6]]

Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.90      0.82        10
           1       0.86      0.67      0.75         9

    accuracy                           0.79        19
   macro avg       0.80      0.78      0.78        19
weighted avg       0.80      0.79      0.79        19

